In [1]:
cd(@__DIR__)
using Pkg; Pkg.activate("."); Pkg.instantiate()

using Revise

using DataFrames, Dates, Statistics
using DelimitedFiles, CSV

using OrdinaryDiffEq, DiffEqFlux, Flux
using SciMLSensitivity

using Optimization, BlackBoxOptim
using Zygote

using Interpolations

using Random
Random.seed!(123)


  Activating project at `~/Documenti/HydroNODE`


TaskLocalRNG()

In [10]:
includet("ae.jl")
data_path = joinpath(pwd(),"basin_dataset_public_v1p2")
data_x, data_sizey, timepoints = prepare_multiple_basins(data_path)

([8.381253 0.0 -11.42; 8.403639 0.0 -10.7; … ; 11.520014 0.0 11.41; 11.424003 0.32 14.075;;;; 8.662769 0.0 -5.655; 8.695014 0.0 -4.54; … ; 11.520002 0.0 14.085; 11.520002 0.0 14.65;;;; 8.542522 0.0 -8.41; 8.56005 0.0 -7.675; … ; 11.520011 0.14 13.24; 11.498417 0.0 14.66;;;; … ;;;; 1.022153f-39 1.012986f-39 1.014457f-39; 0.0 0.0 0.0; … ; 1.012985f-39 1.014456f-39 1.015927f-39; 0.0 0.0 0.0;;;; 1.015929f-39 1.0174f-39 1.018872f-39; 0.0 0.0 0.0; … ; 1.017399f-39 1.01887f-39 1.020341f-39; 0.0 0.0 0.0;;;; 1.020343f-39 1.021814f-39 1.023286f-39; 0.0 0.0 0.0; … ; 1.021813f-39 1.023284f-39 1.014117f-39; 0.0 0.0 0.0], [0.7090447; 0.692807; … ; 0.46006715; 0.42759183;;; 1.6444391; 1.4570979; … ; 0.6785913; 0.699407;;; 1.7327809; 1.5061865; … ; 0.09930167; 0.15061864;;; … ;;; 1.015971f-39; 0.0; … ; 1.017441f-39; 0.0;;; 1.017442f-39; 0.0; … ; 1.018912f-39; 0.0;;; 1.018914f-39; 0.0; … ; 1.020384f-39; 0.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  2090.0, 2091.0, 2092.0, 2093.0, 2094.0, 

In [32]:

function Encoder(enc_space_dim; drop_p=0.5)
    # Define the encoder and decoder networks
    encoder_features = Chain(
        Conv((4, 3), 1 => 8; stride = (4,1), pad = 0),
        BatchNorm(8, leakyrelu),
        Dropout(drop_p),
        
        Conv((4, 1), 8 => 8; stride = 1, pad = 0),
        BatchNorm(8, leakyrelu),
        Dropout(drop_p),
        MaxPool((2, 1), pad=0),
        
        Conv((6, 1), 8 => 16; stride = 1, pad = 0),
        BatchNorm(16, leakyrelu),
        Dropout(drop_p),
        
        Conv((2, 1), 16 => 16; stride = (2,1), pad = 0),
        BatchNorm(16, leakyrelu),
        Dropout(drop_p),
        MaxPool((2, 1), pad=0),
        
        Flux.flatten,
        Dense(16 * 64 * 1, 256, relu),
        Dense(256, enc_space_dim, relu)
    ) |> gpu
    
    return encoder_features
end

enc = Encoder(27)

┌ Info: The GPU function is being called but the GPU is not accessible. 
│ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
└ @ Flux /home/alberto/.julia/packages/Flux/nJ0IB/src/functor.jl:198


Chain(
  Conv((4, 3), 1 => 8, stride=(4, 1)),  # 104 parameters
  BatchNorm(8, leakyrelu),              # 16 parameters, plus 16
  Dropout(0.5),
  Conv((4, 1), 8 => 8),                 # 264 parameters
  BatchNorm(8, leakyrelu),              # 16 parameters, plus 16
  Dropout(0.5),
  MaxPool((2, 1)),
  Conv((6, 1), 8 => 16),                # 784 parameters
  BatchNorm(16, leakyrelu),             # 32 parameters, plus 32
  Dropout(0.5),
  Conv((2, 1), 16 => 16, stride=(2, 1)),  # 528 parameters
  BatchNorm(16, leakyrelu),             # 32 parameters, plus 32
  Dropout(0.5),
  MaxPool((2, 1)),
  Flux.flatten,
  Dense(1024 => 256, relu),             # 262_400 parameters
  Dense(256 => 27, relu),               # 6_939 parameters
)         # Total: 20 trainable arrays, 271_115 parameters,
          # plus 8 non-trainable, 96 parameters, summarysize 1.037 MiB.

In [30]:
loader = Flux.DataLoader((data_x[:,:,:,560],), batchsize=16, shuffle=true);
print(size(data_x))

(2100, 3, 1, 598)

In [31]:


enc(data_x)

27×598 Matrix{Float32}:
 0.588998  0.155452  0.291149  …  4.49724f18  3.1266f-41  0.0
 1.01985   0.118422  0.450991     0.0         2.4703f-41  8.50278f17
 1.64973   1.19627   1.52621      4.57943f18  5.0828f-41  3.34035f18
 1.55591   1.95971   1.6194       1.59424f18  8.8615f-41  9.39955f18
 0.0       0.0       0.0          0.0         0.0         0.0
 0.668317  0.260696  0.0       …  1.4284f19   2.9644f-41  0.0
 0.55157   0.518137  0.777861     1.53912f19  1.2871f-41  5.66696f17
 0.0       0.0       0.0          1.82072f19  0.0         0.0
 0.0       0.0       0.0          0.0         0.0         0.0
 0.0       0.0       0.0          1.21519f19  0.0         0.0
 0.0       0.0       0.0       …  0.0         0.0         2.30659f18
 0.0       0.0       0.0          0.0         0.0         2.44186f17
 0.0       0.0       0.0          1.12855f19  1.38f-42    0.0
 ⋮                             ⋱  ⋮                       
 0.382498  0.0       0.273566  …  1.44663f19  2.1664f-41  2.45676f17


In [ ]:
a = ["2", "3", "4", "5"]
ind = [1,3]
a[ind]

In [ ]:
function ConvEncoder(; imgsize=(365,1,1), nclasses=27) 
    out_conv_size = (imgsize[1]÷4 - 3, 1, 16)
    
    return Chain(
            Conv((5, 1), imgsize[end]=>8, relu),
            MaxPool((2, 1)),
            Conv((5, 1), 6=>16, relu),
            MaxPool((2, 1)),
            flatten,
            Dense(prod(out_conv_size), 128, relu), 
            Dense(128, 64, relu), 
            Dense(64, nclasses)
          )
end